In [1]:
import socket
import time
import random
import numpy as np
from collections import deque

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

###############################
# Параметры подключения и поля
###############################

HOST = 'tetrx-nejehj7lqd2ui.shellweplayaga.me'
PORT = 1989
TICKET = 'ticket{CocoaSnoopy734n25:rZygg5VSdEGmK8edBAQg65osJYxPIaIFkQdkqskQpb1Z_Ifi}'

READ_TIMEOUT = 2.0
RECONNECT_DELAY = 3.0
COMMAND_DELAY = 0.15

# Фиксированный размер игрового поля
TARGET_HEIGHT = 48
TARGET_WIDTH = 16
FEATURE_DIM = TARGET_HEIGHT * TARGET_WIDTH  # 768

# Дополнительный штраф за шаг (чем быстрее линия очищается — тем меньше накопленный отрицательный счет)
STEP_PENALTY = 0.1

# Карта действий: 0:'a', 1:'s', 2:'w', 3:'q', 4:'f', 5:'r'
action_map = {0: 'a', 1: 's', 2: 'w', 3: 'q', 4: 'f', 5: 'r'}
NUM_ACTIONS = 6

###############################
# Функции работы с сокетом
###############################

def read_output(sock, timeout=READ_TIMEOUT):
    sock.settimeout(timeout)
    full_data = b""
    try:
        while True:
            chunk = sock.recv(4096)
            if not chunk:
                break
            full_data += chunk
            if b'Segmentation fault' in full_data or b'Illegal instruction' in full_data or b'flag{' in full_data:
                break
    except socket.timeout:
        pass
    return full_data.decode(errors="ignore")

def send_command(sock, cmd):
    sock.sendall((cmd + "\r\n").encode())
    print(f"[→] Sent: {cmd}")
    time.sleep(COMMAND_DELAY)

###############################
# Функции обработки и нормализации игрового поля
###############################

def parse_field(text):
    """
    Из входного текста ищет строки, состоящие только из символов '.' и 'o'
    (длиной не менее 10 символов) и возвращает самый длинный блок (без нормализации).
    """
    lines = text.splitlines()
    candidate_blocks = []
    current_block = []
    for line in lines:
        l = line.strip()
        if l and set(l) <= {'.', 'o'} and len(l) >= 10:
            current_block.append(l)
        else:
            if current_block:
                candidate_blocks.append(current_block)
                current_block = []
    if current_block:
        candidate_blocks.append(current_block)
    if not candidate_blocks:
        return []
    block = max(candidate_blocks, key=lambda x: len(x))
    return block

def normalize_field(field, target_height=TARGET_HEIGHT, target_width=TARGET_WIDTH):
    """
    Приводит поле (список строк) к размеру target_height x target_width.
    Если строк меньше target_height — добавляет сверху строки с '.',
    если больше – берёт последние target_height строк.
    Каждая строка обрезается или дополняется справа точками.
    """
    if not field:
        return []
    current_height = len(field)
    if current_height < target_height:
        pad = ['.' * target_width] * (target_height - current_height)
        field = pad + field
    elif current_height > target_height:
        field = field[-target_height:]
    field = [row[:target_width].ljust(target_width, '.') for row in field]
    return field

def extract_features(field, target_height=TARGET_HEIGHT, target_width=TARGET_WIDTH):
    """
    Преобразует нормализованное игровое поле в одномерный numpy-вектор длины target_height*target_width.
    Каждая клетка: 1.0, если 'o', 0.0, если '.'.
    """
    norm_field = normalize_field(field, target_height, target_width)
    flat = [1.0 if ch == 'o' else 0.0 for row in norm_field for ch in row]
    return np.array(flat)

def compute_reward(field, target_width=TARGET_WIDTH):
    """
    Вычисляет награду как разницу в количестве полностью заполненных строк в нормализованном поле.
    Если линий не очищено, возвращает -STEP_PENALTY.
    Если очищается одна строка, reward = 10 - STEP_PENALTY (например).
    """
    norm_field = normalize_field(field)
    complete_lines = sum(1 for row in norm_field if set(row) == {'o'})
    return complete_lines * 10 - STEP_PENALTY

def get_best_column(field):
    """
    Для нормализованного поля определяет для каждой колонки номер строки первой заполненной клетки.
    Если колонка пустая, высота = TARGET_HEIGHT.
    Возвращает индекс колонки с максимальным значением.
    """
    norm_field = normalize_field(field)
    width = len(norm_field[0])
    height = len(norm_field)
    heights = [0] * width
    for col in range(width):
        for row in range(height):
            if norm_field[row][col] == "o":
                heights[col] = row
                break
        else:
            heights[col] = height
    best = max(range(width), key=lambda i: heights[i])
    return best

def find_target_column(field):
    """
    Ищет в нормализованном поле строку, где заполнено (TARGET_WIDTH - 1) клетка, начиная снизу.
    Если найдена, возвращает индекс пустой ячейки и флаг True, иначе – get_best_column(field) и False.
    """
    norm_field = normalize_field(field)
    width = len(norm_field[0])
    height = len(norm_field)
    for row in range(height - 1, -1, -1):
        if norm_field[row].count("o") == width - 1:
            target = norm_field[row].index(".")
            print(f"[DEBUG] Найдена почти заполненная строка на row={row}, пустой индекс: {target}")
            return target, True
    return get_best_column(field), False

def move_to_column(current_col, target_col):
    cmds = []
    while current_col > target_col:
        cmds.append("a")
        current_col -= 1
    while current_col < target_col:
        cmds.append("s")
        current_col += 1
    return cmds

###############################
# DQN-нейросетевая модель и агент
###############################

class DQNNet(nn.Module):
    def __init__(self, input_height, input_width, num_actions):
        super(DQNNet, self).__init__()
        # Входное изображение: 1 канал, размер (input_height x input_width)
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc_input_dim = 64 * input_height * input_width
        self.fc1 = nn.Linear(self.fc_input_dim, 512)
        self.fc2 = nn.Linear(512, num_actions)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

class ReplayBuffer:
    def __init__(self, capacity=10000):
        self.buffer = deque(maxlen=capacity)

    def push(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)
        return (np.array(states),
                np.array(actions),
                np.array(rewards, dtype=np.float32),
                np.array(next_states),
                np.array(dones, dtype=np.float32))

    def __len__(self):
        return len(self.buffer)

class DQNAgent:
    def __init__(self, input_height, input_width, num_actions, lr=1e-3, gamma=0.99,
                 epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.1, batch_size=32):
        self.num_actions = num_actions
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.batch_size = batch_size

        self.model = DQNNet(input_height, input_width, num_actions)
        self.target_model = DQNNet(input_height, input_width, num_actions)
        self.update_target()
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        self.replay_buffer = ReplayBuffer(10000)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
        self.target_model.to(self.device)

    def update_target(self):
        self.target_model.load_state_dict(self.model.state_dict())

    def select_action(self, state):
        if np.random.rand() < self.epsilon:
            action_idx = np.random.choice(self.num_actions)
            print(f"[AGENT] Случайный выбор: {action_map[action_idx]}")
            return action_idx
        state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device)
        state_tensor = state_tensor.view(1, 1, TARGET_HEIGHT, TARGET_WIDTH)
        with torch.no_grad():
            q_values = self.model(state_tensor)
        action_idx = q_values.argmax().item()
        print(f"[AGENT] Выбор по оценке: {action_map[action_idx]} (q_values={q_values.cpu().numpy()})")
        return action_idx

    def push_experience(self, state, action, reward, next_state, done):
        self.replay_buffer.push(state, action, reward, next_state, done)

    def train_step(self):
        if len(self.replay_buffer) < self.batch_size:
            return None
        states, actions, rewards, next_states, dones = self.replay_buffer.sample(self.batch_size)
        states = torch.FloatTensor(states).to(self.device).view(self.batch_size, 1, TARGET_HEIGHT, TARGET_WIDTH)
        next_states = torch.FloatTensor(next_states).to(self.device).view(self.batch_size, 1, TARGET_HEIGHT, TARGET_WIDTH)
        actions = torch.LongTensor(actions).to(self.device)
        rewards = torch.FloatTensor(rewards).to(self.device)
        dones = torch.FloatTensor(dones).to(self.device)

        q_values = self.model(states)
        q_value = q_values.gather(1, actions.unsqueeze(1)).squeeze(1)
        next_q_values = self.target_model(next_states)
        next_q_value = next_q_values.max(1)[0]
        expected_q_value = rewards + self.gamma * next_q_value * (1 - dones)

        loss = F.mse_loss(q_value, expected_q_value.detach())
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        return loss.item()

###############################
# Основная логика работы бота с DQN агентом
###############################

def run_bot():
    with socket.create_connection((HOST, PORT)) as s:
        print("[+] Connected")
        greeting = read_output(s, timeout=RECONNECT_DELAY)
        print("[SERVER]:", greeting)
        s.sendall((TICKET + "\r\n").encode())
        print("[+] Sent ticket")

        prev_complete = 0  # Количество полных (очищенных) строк в предыдущем состоянии

        while True:
            out = read_output(s, timeout=READ_TIMEOUT)
            print("[SERVER]:\n", out)
            lower_out = out.lower()
            if "flag{" in lower_out:
                print("[🏁] FLAG FOUND!")
                break
            if "segmentation fault" in lower_out or "illegal instruction" in lower_out:
                raise Exception("Server error detected!")

            parsed_field = parse_field(out)
            if not parsed_field:
                print("[-] Не удалось распарсить поле. [DEBUG RAW OUTPUT]:")
                print(out)
                send_command(s, "w")
                continue

            norm_field = normalize_field(parsed_field)
            print("[FIELD]:")
            for line in norm_field:
                print(line)

            state = extract_features(parsed_field, TARGET_HEIGHT, TARGET_WIDTH)
            if state.shape[0] != FEATURE_DIM:
                print(f"[DEBUG] Неверная размерность признаков: {state.shape[0]} (ожидается {FEATURE_DIM})")

            action_idx = agent.select_action(state)
            action_cmd = action_map[action_idx]
            send_command(s, action_cmd)

            new_out = read_output(s, timeout=READ_TIMEOUT)
            new_parsed_field = parse_field(new_out)
            if not new_parsed_field:
                continue
            next_state = extract_features(new_parsed_field, TARGET_HEIGHT, TARGET_WIDTH)
            new_complete = compute_reward(new_parsed_field, TARGET_WIDTH) / 10.0  # делим на 10, так как бонус за линию * 10
            # Изменяем награду: разница между новым и предыдущим числом полных линий, минус штраф за шаг
            reward = (new_complete - prev_complete) - STEP_PENALTY
            prev_complete = new_complete
            print(f"[REWARD] Получена награда: {reward}")

            agent.push_experience(state, action_idx, reward, next_state, 0.0)
            loss = agent.train_step()
            if loss is not None:
                print(f"[LOSS] {loss:.4f}")
            if random.random() < 0.05:
                agent.update_target()
                print("[TARGET] Обновление целевой сети.")

def main():
    while True:
        try:
            run_bot()
            break
        except Exception as e:
            print(f"[!] Exception: {e}")
            print(f"[!] Переподключаемся через {RECONNECT_DELAY} секунд...")
            time.sleep(RECONNECT_DELAY)

###############################
# Инициализация DQN агента
###############################

agent = DQNAgent(TARGET_HEIGHT, TARGET_WIDTH, NUM_ACTIONS, lr=1e-3, gamma=0.99,
                   epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.1, batch_size=32)

if __name__ == "__main__":
    main()

[+] Connected
[SERVER]: Ticket please: 
[+] Sent ticket
[SERVER]:
 Debug: 
[-] Не удалось распарсить поле. [DEBUG RAW OUTPUT]:
Debug: 
[→] Sent: w
[SERVER]:
 Seed: ................
.......oo.......
.......oo.......
................
................
................
................
................
................
................
................
................
................
................
................
................
................
................
................
................
................
................
................
................
........o.......
......ooo.......
................
................
................
................
................
................
................
................
................
................
................
................
................
................
................
................
................
................
................
................
.......oo.......
.......oo.......

[FIELD]:
..........

KeyboardInterrupt: 